In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import re
from pandas import ExcelWriter
import xlsxwriter
from datetime import datetime
import pandas_datareader.data as wb

In [2]:
kospi = pd.read_excel('/workspace/Jupyter_notebook/financial_analysis/data/kospi.xls')

In [3]:
kospi

,번호,종목코드,기업명,업종코드,업종,상장주식수(주),자본금(원),액면가(원),통화구분,대표전화,주소
0,1,95570,AJ네트웍스,147603,산업용 기계 및 장비 임대업,"46,822,295","46,822,295,000","1,000",원(KRW),02-6363-9999,"서울특별시 송파구 정의로8길 9 (문정동,AJ빌딩)"
1,2,6840,AK홀딩스,116409,기타 금융업,"13,247,561","66,237,805,000","5,000",원(KRW),02-768-2923,서울특별시 마포구 양화로 188 -
2,3,27410,BGF,116409,기타 금융업,"95,716,791","95,716,791,000","1,000",NaN,1577-3663,서울특별시 강남구 테헤란로 405
3,4,282330,BGF리테일,74701,종합 소매업,"17,283,906","17,283,906,000","1,000",원(KRW),02-1577-8007,서울특별시 강남구 테헤란로 405 BGF사옥
4,5,138930,BNK금융지주,116409,기타 금융업,"325,935,246","1,629,676,230,000","5,000",원(KRW),051-620-3023,부산광역시 남구 문현금융로 30(문현동)
...,...,...,...,...,...,...,...,...,...,...,...
795,796,79980,휴비스,32005,화학섬유 제조업,"34,500,000","172,500,000,000","5,000",원(KRW),02-2189-4567,서울특별시 강남구 학동로 343
796,797,5010,휴스틸,32401,1차 철강 제조업,"7,621,668","38,108,340,000","5,000",원(KRW),02-828-9000,"서울특별시 강남구 테헤란로 512 (대치동, 신안빌딩 14층)"
797,798,69260,휴켐스,32004,기타 화학제품 제조업,"40,878,588","40,878,588,000","1,000",원(KRW),02-2262-0600,서울특별시 중구 퇴계로 173 19층
798,799,540,흥국화재,116501,보험업,"64,242,645","325,821,225,000","5,000",원(KRW),02-2002-6000,서울특별시 종로구 새문안로 68


In [4]:
stock_code_list = []
def two_len(code):
    code = '0000' + code
    return code

def three_len(code):
    code = '000' + code
    return code

def four_len(code):
    code = '00' + code
    return code
    
def five_len(code):
    code = '0' + code
    return code

for i in range(0, len(kospi['종목코드'])):
    if(len(str(kospi['종목코드'][i])) == 4):
        stock_code_list.append([four_len(str(kospi['종목코드'][i])), kospi['기업명'][i]])
    elif(len(str(kospi['종목코드'][i])) == 5):
        stock_code_list.append([five_len(str(kospi['종목코드'][i])), kospi['기업명'][i]])
    elif(len(str(kospi['종목코드'][i])) == 3):
        stock_code_list.append([three_len(str(kospi['종목코드'][i])), kospi['기업명'][i]])
    elif(len(str(kospi['종목코드'][i])) == 2):
        stock_code_list.append([two_len(str(kospi['종목코드'][i])), kospi['기업명'][i]])
    else:
        stock_code_list.append([str(kospi['종목코드'][i]), kospi['기업명'][i]])
    

In [5]:
stock_code_list

[['095570', 'AJ네트웍스'],
 ['006840', 'AK홀딩스'],
 ['027410', 'BGF'],
 ['282330', 'BGF리테일'],
 ['138930', 'BNK금융지주'],
 ['001460', 'BYC'],
 ['001040', 'CJ'],
 ['079160', 'CJ CGV'],
 ['000120', 'CJ대한통운'],
 ['011150', 'CJ씨푸드'],
 ['097950', 'CJ제일제당'],
 ['000590', 'CS홀딩스'],
 ['012030', 'DB'],
 ['016610', 'DB금융투자'],
 ['005830', 'DB손해보험'],
 ['000990', 'DB하이텍'],
 ['139130', 'DGB금융지주'],
 ['004840', 'DRB동일'],
 ['155660', 'DSR'],
 ['069730', 'DSR제강'],
 ['017940', 'E1'],
 ['007700', 'F&F'],
 ['114090', 'GKL'],
 ['078930', 'GS'],
 ['006360', 'GS건설'],
 ['001250', 'GS글로벌'],
 ['007070', 'GS리테일'],
 ['012630', 'HDC'],
 ['039570', 'HDC아이콘트롤스'],
 ['089470', 'HDC현대EP'],
 ['294870', 'HDC현대산업개발'],
 ['082740', 'HSD엔진'],
 ['175330', 'JB금융지주'],
 ['234080', 'JW생명과학'],
 ['001060', 'JW중외제약'],
 ['096760', 'JW홀딩스'],
 ['105560', 'KB금융'],
 ['009440', 'KC그린홀딩스'],
 ['119650', 'KC코트렐'],
 ['092220', 'KEC'],
 ['001390', 'KG케미칼'],
 ['001940', 'KISCO홀딩스'],
 ['025000', 'KPX케미칼'],
 ['092230', 'KPX홀딩스'],
 ['000040', 'KR모터스'],
 ['0444

In [6]:
stock_code = pd.DataFrame(stock_code_list, columns=['종목코드','기업명'])

In [7]:
stock_code

,종목코드,기업명
0,095570,AJ네트웍스
1,006840,AK홀딩스
2,027410,BGF
3,282330,BGF리테일
4,138930,BNK금융지주
...,...,...
795,079980,휴비스
796,005010,휴스틸
797,069260,휴켐스
798,000540,흥국화재


In [8]:
def make_excel(df, company):
    file_name = '/workspace/Jupyter_notebook/financial_analysis/data/stock/'+company+'.xlsx'
    writer = pd.ExcelWriter(file_name, engine='xlsxwriter')
    df.to_excel(writer, sheet_name='company_list')
    writer.close()

In [ ]:
stock = pd.DataFrame()
pattern = re.compile('(\d+)')
for i in range(29, len(stock_code['종목코드'])):
    url = "https://finance.naver.com/item/sise_day.nhn?code={}&page={}"
    try:
        last = pattern.findall(bs(requests.get(url.format(stock_code['종목코드'][i], 1)).text, 'html.parser').find("td", class_='pgRR').find("a")['href'])[-1]
    except:
        last = 1
    for cnt in range(1, int(last)+1):
    #for cnt in range(1, 2):
        #print (cnt)
        data = pd.read_html(url.format(stock_code['종목코드'][i], cnt))[0].dropna()
        
        # 데이터가 중복되어 있다면 primary key 값으로 인하여 오류가 발생한다.
        stock = stock.append(data)
    
    stock.reset_index(drop = True, inplace = True)
    make_excel(stock, stock_code['기업명'][i])

In [ ]:
#종목하나 테스트
# stock = pd.DataFrame()
# pattern = re.compile('(\d+)')
# url = "https://finance.naver.com/item/sise_day.nhn?code={}&page={}"
# try:
#     last = pattern.findall(bs(requests.get(url.format(stock_code['종목코드'][0], 1)).text, 'html.parser').find("td", class_='pgRR').find("a")['href'])[-1]
# except:
#     last = 1
# for cnt in range(1, int(last)+1):
# #for cnt in range(1, 2):
#     #print (cnt)
#     data = pd.read_html(url.format(stock_code['종목코드'][0], cnt))[0].dropna()
#     stock = stock.append(data)

# stock.reset_index(drop = True, inplace = True)


In [ ]:
stock.reset_index(drop = True, inplace = True)

In [ ]:
stock